In [5]:
%load_ext autoreload
%autoreload 2

import pandas as pd
import numpy as np
import utm
import warnings
warnings.filterwarnings('ignore')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [127]:
path = r'C:\Users\NiFa\Desktop\Data Science\CapStone\data\Ride_Data_Cache'
order_data = pd.read_msgpack(r'C:\Users\NiFa\Desktop\Data Science\CapStone\data\Ride_Data_Cache\order_20161102.msgpack')
# gps_data = pd.read_msgpack(r'C:\Users\NiFa\Desktop\gps_data_utm_nov2.msgpack')

In [68]:
gps_data.head()

,driver_id,order_id,timestamp,longitude,latitude,utm_x,utm_y
0,a739b90e4907fa30b0d6a3a3b39e67bb,982bf243c3202415d6252271b2693161,1478041352,104.10056,30.70907,413872.724770,3.397706e+06
1,a739b90e4907fa30b0d6a3a3b39e67bb,982bf243c3202415d6252271b2693161,1478041355,104.10040,30.70882,413857.180940,3.397678e+06
2,a739b90e4907fa30b0d6a3a3b39e67bb,982bf243c3202415d6252271b2693161,1478041358,104.10017,30.70846,413834.835993,3.397638e+06
3,a739b90e4907fa30b0d6a3a3b39e67bb,982bf243c3202415d6252271b2693161,1478041361,104.10001,30.70822,413819.300864,3.397612e+06
4,a739b90e4907fa30b0d6a3a3b39e67bb,982bf243c3202415d6252271b2693161,1478041364,104.09991,30.70806,413809.582479,3.397594e+06


In [69]:
order_data.head()

,order_id,ride_start_timestamp,ride_stop_timestamp,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude
0,39d471630d26439111a7746d164d34eb,1478091677,1478092890,104.008160,30.706220,104.064147,30.685848
1,6681082cbcc11b37842489a096e2172e,1478093722,1478094806,104.053993,30.684104,104.015670,30.633230
2,56728c3d9555a452307d47986a773d9d,1478081351,1478082003,104.068120,30.708790,104.049720,30.704970
3,5b2724a9fd290d0388264fe07a9aec4c,1478082542,1478083418,104.059534,30.696013,104.034630,30.720960
4,0a2055d9c863e2f1963856d4d305ccd9,1478078909,1478080807,104.079048,30.796888,104.083284,30.705019


In [128]:
temp_pickup = order_data.apply(lambda x: utm.from_latlon(x['pickup_latitude'], x['pickup_longitude'])[0:2], axis=1)
temp_drop = order_data.apply(lambda x: utm.from_latlon(x['dropoff_latitude'], x['dropoff_longitude'])[0:2], axis=1)

In [129]:
order_data['xpickup'] = temp_pickup.apply(lambda x: x[0])
order_data['ypickup'] = temp_pickup.apply(lambda x: x[1])

order_data['xdropoff'] = temp_drop.apply(lambda x: x[0])
order_data['ydropoff'] = temp_drop.apply(lambda x: x[1])

In [130]:
orig_data = order_data

In [131]:
grid_x_num = 40
grid_y_num = 40

In [132]:
tempx = pd.cut(order_data['xpickup'],bins = grid_x_num).astype(str)
tempy = pd.cut(order_data['ypickup'],bins = grid_y_num).astype(str)
order_data['pick_up_zone'] = tempx + tempy

In [133]:
tempx = pd.cut(order_data['xdropoff'],bins = grid_x_num).astype(str)
tempy = pd.cut(order_data['ydropoff'],bins = grid_y_num).astype(str)
order_data['drop_off_zone'] = tempx + tempy

In [134]:
drivers = gps_data[['driver_id', 'order_id']].drop_duplicates()
order_data  = order_data.merge(drivers, on ='order_id', how='left')

In [135]:
order_data.head()

,order_id,ride_start_timestamp,ride_stop_timestamp,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,xpickup,ypickup,xdropoff,ydropoff,pick_up_zone,drop_off_zone,driver_id
0,39d471630d26439111a7746d164d34eb,1478091677,1478092890,104.008160,30.706220,104.064147,30.685848,405021.620476,3.397464e+06,410364.332827,3.395160e+06,"(404880.67, 407264.234](3396021.364, 3397779.732]","(409718.164, 417579.349](3393567.88, 3480582.442]",af17fba73f06db61f7596163eac1eb73
1,6681082cbcc11b37842489a096e2172e,1478093722,1478094806,104.053993,30.684104,104.015670,30.633230,409390.115174,3.394975e+06,405669.915408,3.389369e+06,"(407264.234, 409647.798](3394262.995, 3396021....","(401856.98, 409718.164](3306553.319, 3393567.88]",af17fba73f06db61f7596163eac1eb73
2,56728c3d9555a452307d47986a773d9d,1478081351,1478082003,104.068120,30.708790,104.049720,30.704970,410765.994637,3.397700e+06,409000.403342,3.397291e+06,"(409647.798, 412031.361](3396021.364, 3397779....","(401856.98, 409718.164](3393567.88, 3480582.442]",ae8221abe491455c7c0bedb0f00d6e8a
3,5b2724a9fd290d0388264fe07a9aec4c,1478082542,1478083418,104.059534,30.696013,104.034630,30.720960,409931.921294,3.396291e+06,407570.558398,3.399076e+06,"(409647.798, 412031.361](3396021.364, 3397779....","(401856.98, 409718.164](3393567.88, 3480582.442]",ae8221abe491455c7c0bedb0f00d6e8a
4,0a2055d9c863e2f1963856d4d305ccd9,1478078909,1478080807,104.079048,30.796888,104.083284,30.705019,411892.725646,3.407455e+06,412214.698706,3.397270e+06,"(409647.798, 412031.361](3406571.575, 3408329....","(409718.164, 417579.349](3393567.88, 3480582.442]",ae8221abe491455c7c0bedb0f00d6e8a


In [146]:
len(order_data['pick_up_zone'].unique())

453

In [182]:
temp = order_data.groupby(['driver_id', 'pick_up_zone']).count()/ order_data.groupby(['driver_id']).count()

In [183]:
temp = temp['order_id']

In [184]:
temp.head()

driver_id                         pick_up_zone                                      
000203e2b9574dec0f9e22e1471d7343  (404880.67, 407264.234](3410088.312, 3411846.681]     1.0
00020e3d875210ba7b86de9a17a969d9  (407264.234, 409647.798](3388987.889, 3390746.258]    0.4
                                  (409647.798, 412031.361](3392504.627, 3394262.995]    0.2
                                  (412031.361, 414414.925](3388987.889, 3390746.258]    0.4
0002a9da2932f75a1d5f12fc99ea9945  (409647.798, 412031.361](3392504.627, 3394262.995]    0.3
Name: order_id, dtype: float64

In [169]:
temp.head()

driver_id                         pick_up_zone                                      
000203e2b9574dec0f9e22e1471d7343  (404880.67, 407264.234](3410088.312, 3411846.681]     1
00020e3d875210ba7b86de9a17a969d9  (407264.234, 409647.798](3388987.889, 3390746.258]    2
                                  (409647.798, 412031.361](3392504.627, 3394262.995]    1
                                  (412031.361, 414414.925](3388987.889, 3390746.258]    2
0002a9da2932f75a1d5f12fc99ea9945  (409647.798, 412031.361](3392504.627, 3394262.995]    6
Name: order_id, dtype: int64